### recuperation des données, initiaisation, et premier clustering hierarchique

In [20]:
import pandas as pd
import numpy as np
from sklearn.cluster import AgglomerativeClustering,KMeans, AffinityPropagation, SpectralClustering, MeanShift
from shapely.geometry import mapping, shape, Polygon, MultiPoint, Point
from sklearn.neighbors import kneighbors_graph

datas = pd.read_csv("../data/resultats_api.csv")
datas.head(5)

,Unnamed: 0,trip_completed_at,trip_status,ride_hailing_app,trip_uid,driver_uid,rider_uid,customer,trip_start_time,trip_end_time,...,temperature_value,feels_like,humidity,wind_speed,cloudness,weather_main,weather_desc,precipitation,place_type,place_name
0,0,"May 11, 2015 at 6:55PM",Completed,Uber,ee89076fd9da9bddf5f096b0ca42f8d5,05cfeb269e606247fe9d2b6082942c59,3ffa4a71a5aa791a8bc3409f5b15b936,stantyan,2015-05-11 17:26:00,2015-05-11 17:55:00,...,12,12,0.69,4.81,0.78,partly-cloudy-day,Mostly Cloudy,none,route restaurant food point_of_interest estab...,"Bolshoy Prospekt, 74 Capuletti Stomadeus Agen..."
1,1,"May 11, 2015 at 8:12PM",Completed,Uber,518be51d403944a03c47e8d1f2c87311,4a4e248742f9d5ff517c5bbbb48d0e54,3ffa4a71a5aa791a8bc3409f5b15b936,stantyan,2015-05-11 18:46:00,2015-05-11 19:12:00,...,10,10,0.70,6.53,0.66,partly-cloudy-day,Mostly Cloudy,none,locality political point_of_interest establis...,Saint Petersburg Club Music balcony Beauty Sa...
2,2,"May 13, 2015 at 11:38AM",Completed,Uber,6e460cc8a12c3c6568d0d4a67ac58393,cb249a2bd807ca78697b4ed0348c37da,3ffa4a71a5aa791a8bc3409f5b15b936,stantyan,2015-05-13 09:15:00,2015-05-13 10:38:00,...,14,14,0.61,5.25,0.86,partly-cloudy-day,Mostly Cloudy,none,route point_of_interest store establishment r...,Nevsky prospect Sofrino Империя+ Business Por...
3,3,"May 16, 2015 at 1:44AM",Completed,Uber,49613a86a04e6c15d72b51d1a2935d81,d3f73f8151c2e8c34b541f961db7f5fa,3ffa4a71a5aa791a8bc3409f5b15b936,stantyan,2015-05-16 00:24:00,2015-05-16 00:44:00,...,3,3,0.84,0.87,0.56,partly-cloudy-night,Partly Cloudy,none,route lodging point_of_interest establishment...,"Voznesensky Ave, 14 Piter Bridges Dostup Na K..."
4,4,"May 16, 2015 at 3:18AM",Completed,Uber,9896148fdecdb4c5d977a8691510bdb6,1287d21e6455ee40d4861f6b91c680f4,3ffa4a71a5aa791a8bc3409f5b15b936,stantyan,2015-05-16 01:29:00,2015-05-16 02:18:00,...,3,1,0.90,1.61,0.31,partly-cloudy-night,Partly Cloudy,none,route pet_store point_of_interest store estab...,"Ulitsa Yesenina Zoomagazin Stomatologiya ""Gal..."


In [27]:
d = datas[['pickup_lat','pickup_long','dropoff_lat','dropoff_long','trip_start_time','trip_end_time']].values

coords=np.empty([0,3])
for i in d:
    coords= np.append(coords, [[i[0],i[1],i[4]]],axis=0)
    coords = np.append(coords, [[i[2],i[3],i[5]]],axis=0)

neighb_mtx = coords[:,:2]
neighb_mtx = neighb_mtx.astype(np.float64)


coords

array([['60.03143761', '30.32982593', '2015-05-11 17:26:00'],
       ['59.96313143', '30.30765496', '2015-05-11 17:55:00'],
       ['59.96301417', '30.30731265', '2015-05-11 18:46:00'],
       ...,
       ['59.941415', '30.366456300000007', '2018-04-26 17:00:00'],
       ['59.941415', '30.366456300000007', '2018-04-27 14:15:00'],
       ['59.955692000000006', '30.35657', '2018-04-27 15:05:00']],
      dtype='<U32')

In [3]:


hierarchical = AgglomerativeClustering(n_clusters=20, linkage='single')#,connectivity=neighbors)
result = hierarchical.fit_predict(neighb_mtx)


In [4]:
result = result.reshape(result.shape[0], 1)
result.shape
coords_final = np.append(coords, result, axis=1)
coords_final.shape
coords_final

array([['59.96313143', '30.30765496', '2015-05-11 17:55:00', '1'],
       ['60.03135115', '30.32949527', '2015-05-11 19:12:00', '1'],
       ['59.92428100000001', '30.3875608', '2015-05-13 10:38:00', '1'],
       ...,
       ['59.91683199999999', '30.450724', '2018-04-26 15:57:00', '1'],
       ['59.941415', '30.366456300000007', '2018-04-26 17:00:00', '1'],
       ['59.955692000000006', '30.35657', '2018-04-27 15:05:00', '1']],
      dtype='<U32')

In [5]:
csv_final = pd.DataFrame(coords_final, columns=['lat','long','trip_time', 'cluster'])
csv_final.to_csv(path_or_buf="../data/clustering_hierarchique_20c.csv",header=True, index=True )

La methode actuelle ne permet que de faire des approximation à l'echelle du pays, on peut seulement déduire la ville ou le sujet réside de façon assez grossière.

### Limitation de la zone d'étude sur St petersbourg et ré exécution de l'algorithme de clustering hierarchique

In [6]:
top_left = [60.212733, 29.367030]
bottom_left =[59.681719, 29.893709]
top_right=[60.160548, 30.737993]
bottom_right=[59.613360, 30.80276]

In [7]:
polygonStPetersb = Polygon([(top_left), (bottom_left), (bottom_right), (top_right)])



In [8]:
coords_poly = np.empty([0,3])
for i in d:
    if polygonStPetersb.contains(Point(i[0],i[1])):
        coords_poly = np.append(coords_poly,[[i[0],i[1],i[4]]],axis=0 )
    if  polygonStPetersb.contains(Point(i[2],i[3])):
        coords_poly = np.append(coords_poly,[[i[2],i[3],i[5]]],axis=0 )

coords_poly

array([['60.03143761', '30.32982593', '2015-05-11 17:26:00'],
       ['59.96313143', '30.30765496', '2015-05-11 17:55:00'],
       ['59.96301417', '30.30731265', '2015-05-11 18:46:00'],
       ...,
       ['59.941415', '30.366456300000007', '2018-04-26 17:00:00'],
       ['59.941415', '30.366456300000007', '2018-04-27 14:15:00'],
       ['59.955692000000006', '30.35657', '2018-04-27 15:05:00']],
      dtype='<U32')

In [9]:
neighbors = kneighbors_graph(coords_poly[:,:2].astype(np.float64), 10,include_self=True, mode='connectivity').astype(np.float64)

In [10]:
hierarchical.n_clusters = 15
hierarchical.connectivity = neighbors

result_polygon = hierarchical.fit_predict(coords_poly[:,:2].astype(np.float64))
result_polygon = result_polygon.reshape(result_polygon.shape[0], 1)

/home/vincent/anaconda3/lib/python3.8/site-packages/sklearn/cluster/_agglomerative.py:500: UserWarning: the number of connected components of the connectivity matrix is 2 > 1. Completing it to avoid stopping the tree early.
  connectivity, n_connected_components = _fix_connectivity(


In [11]:
coords_poly_final = np.append(coords_poly, result_polygon, axis=1)
coords_poly_final.shape
coords_poly_final

array([['60.03143761', '30.32982593', '2015-05-11 17:26:00', '1'],
       ['59.96313143', '30.30765496', '2015-05-11 17:55:00', '1'],
       ['59.96301417', '30.30731265', '2015-05-11 18:46:00', '1'],
       ...,
       ['59.941415', '30.366456300000007', '2018-04-26 17:00:00', '1'],
       ['59.941415', '30.366456300000007', '2018-04-27 14:15:00', '1'],
       ['59.955692000000006', '30.35657', '2018-04-27 15:05:00', '1']],
      dtype='<U32')

In [12]:

csv_final = pd.DataFrame(coords_poly_final, columns=['lat','long','trip_time', 'cluster'])
csv_final.to_csv(path_or_buf="../data/clustering_polygonLimited_15c.csv",header=True, index=True )


### Affinity propagation
    Cet algorithme de clustering permet d'ignorer la spécification du nombre de clusters dans les parametres
    Il fonctionne sur une méthode de centroides et détermine les centres des clusters qui ont été trouvés; cela pourrait permettre de localiser plus précisément un quartier potentiellement importants et parmi ceux-là le lieu de résidence.
    De plus, il est bien adapté aux petits / moyens datasets comme le notre.

In [13]:
neighb_affinity = kneighbors_graph(coords_poly[:,:2].astype(np.float64), mode="distance", n_neighbors=10)

In [14]:
affinity_cluster = AffinityPropagation(damping= 0.7,random_state=1 )

result_propag=affinity_cluster.fit_predict(neighb_affinity)#coords_poly[:,:2].astype(np.float64))

result_propag = result_propag.reshape(result_propag.shape[0], 1)
coords_affinity = np.append(coords_poly[:,:2], result_propag, axis=1)
coords_affinity

array([['60.03143761', '30.32982593', '3'],
       ['59.96313143', '30.30765496', '0'],
       ['59.96301417', '30.30731265', '0'],
       ...,
       ['59.941415', '30.366456300000007', '109'],
       ['59.941415', '30.366456300000007', '109'],
       ['59.955692000000006', '30.35657', '169']], dtype='<U32')

In [15]:
csv_affinity = pd.DataFrame(coords_affinity, columns=['lat','long','cluster'])
csv_affinity.to_csv(path_or_buf="../data/clustering_with_nclusters_pre_defined1.csv",header=True, index=True )

### Spectral Clutering 
        Ici on teste le clustering spectral notament parce que cet algorithme se sert d'une matrice d'affinité / distance pour effectuer plusieurs fois un clustering KMeans sur des sous-parties du graphe ou de la matrice passé(e) en paramètre

In [16]:
spectral = SpectralClustering(n_clusters= 10, random_state=1, affinity='rbf', assign_labels='discretize')
result_spect = spectral.fit_predict(coords_poly[:,:2].astype(np.float64))
result_spect= result_spect.reshape(result_spect.shape[0],1)


In [17]:
data_spect = np.hstack((coords_poly, result_spect))
data_spect


array([['60.03143761', '30.32982593', '2015-05-11 17:26:00', '1'],
       ['59.96313143', '30.30765496', '2015-05-11 17:55:00', '9'],
       ['59.96301417', '30.30731265', '2015-05-11 18:46:00', '9'],
       ...,
       ['59.941415', '30.366456300000007', '2018-04-26 17:00:00', '0'],
       ['59.941415', '30.366456300000007', '2018-04-27 14:15:00', '0'],
       ['59.955692000000006', '30.35657', '2018-04-27 15:05:00', '0']],
      dtype='<U32')

In [18]:
csv_spect = pd.DataFrame(data_spect, columns=['lat','long','trip_time', 'cluster'])
csv_spect.to_csv(path_or_buf="../data/SpectralClustering_10c_out.csv",header=True, index=True )

### MeanShift 
    Algorithme de clstering basé ur un principe de centroide, ne dépend pas forcément d'une valeur stipulant le nombre de clusters souhaités. Cet algorithme permet de découvrir des clusters dans un jeu de données régulier.

In [25]:
meanshift = MeanShift(bandwidth=2, max_iter=500, cluster_all=False, min_bin_freq=5)
result_shift = meanshift.fit_predict(coords_poly[:,:2].astype(np.float64))

result_shift = result_shift.reshape(result_shift.shape[0], 1)

In [26]:
datas_meanshift = np.hstack((coords_poly, result_shift))

meanshift_df = pd.DataFrame(coords_poly_final, columns=['lat','long','trip_time', 'cluster'])
meanshift_df.to_csv(path_or_buf="../data/Clustering_MeanShift.csv",header=True, index=True )

### Isolation des plages temporelles 

Ici on isole les plages de temps dans la journée ou le "sujet" est le plus propice à prendre un uber pour partir / rentrer chez lui , cad tard le soir ou les transports ne fonctionnent plus ou bien tôt le matin pour se rendre à l'aeroport par exemple. 


##### Isolation temporelle 


In [29]:
trajet_horaires_soiree= datas[['dropoff_lat', 'dropoff_long', 'trip_end_time']].values
trajet_horaires_matin = datas[['pickup_lat','pickup_long', 'trip_start_time']].values
horaires = np.vstack((trajet_horaires_soiree, trajet_horaires_matin))
trajet_horaires_soiree
trajet_horaires_matin.shape
horaires.shape

(1356, 3)

In [42]:
def get_horaires(soir, tab):
    tab_h=np.ndarray(shape=(0,3))
    
    for i in tab:
        
        heure = int(i[-1].split(' ')[-1].split(':')[0])
        if polygonStPetersb.contains(Point(i[0],i[1])):
            if soir:
                if (heure >= 22 or heure <= 6):
                    tab_h = np.vstack((tab_h, i))
            else:
                if heure >= 5 and heure < 10:
                    tab_h = np.vstack((tab_h, i))
    
    return tab_h
            

trajets = get_horaires(True, horaires)
matin = get_horaires(False, trajet_horaires_matin)


soiree.ndim

2

In [43]:


spectral.n_clusters=4

res_trajets = spectral.fit_predict(trajets[:,:2])
res_trajets = res_trajets.reshape(res_trajets.shape[0], 1)
print(res_trajet)
trajets = np.hstack((trajets, res_trajets))
trajets

[[1]
 [1]
 [1]
 [7]
 [1]
 [1]
 [1]
 [1]
 [7]
 [7]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [5]
 [8]
 [5]
 [3]
 [7]
 [0]
 [8]
 [8]
 [3]
 [7]
 [7]
 [7]
 [9]
 [7]
 [5]
 [5]
 [5]
 [5]
 [5]
 [1]
 [0]
 [8]
 [1]
 [8]
 [9]
 [2]
 [5]
 [1]
 [0]
 [1]
 [2]
 [1]
 [1]
 [6]
 [1]
 [9]
 [0]
 [1]
 [6]
 [1]
 [8]
 [1]
 [1]
 [7]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [7]
 [0]
 [9]
 [9]
 [5]
 [6]
 [5]
 [0]
 [7]
 [9]
 [1]
 [8]
 [8]
 [8]
 [7]
 [7]
 [7]
 [7]
 [5]
 [8]
 [1]
 [1]
 [2]
 [4]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [2]
 [1]
 [2]
 [1]]


array([[59.93467988, 30.30848929, '2015-05-16 00:44:00', 1],
       [60.0314703, 30.3294021, '2015-05-16 02:18:00', 1],
       [60.03133570000001, 30.329517600000006, '2015-05-18 22:08:00', 1],
       [59.79990266, 30.27323585, '2015-06-14 06:44:00', 3],
       [59.93286, 30.3456956, '2015-06-14 22:19:00', 1],
       [59.9277426, 30.3379478, '2015-06-14 23:46:00', 1],
       [59.91925500000001, 30.2846216, '2015-06-15 22:52:00', 1],
       [59.92764293, 30.33816057, '2015-06-17 05:59:00', 1],
       [59.8423166, 30.319145, '2015-07-08 01:16:00', 3],
       [59.84233031, 30.31906726, '2015-07-09 01:33:00', 3],
       [60.01617182, 30.40927137, '2015-09-16 22:47:00', 1],
       [60.01617823, 30.40939246, '2015-09-28 22:49:00', 1],
       [60.0161933, 30.4092933, '2015-10-20 06:40:00', 1],
       [60.01613255, 30.40924195, '2015-11-28 22:36:00', 1],
       [60.0161323, 30.409212100000005, '2016-06-12 23:57:00', 1],
       [60.0162533, 30.4092333, '2016-06-13 23:42:00', 1],
       [59.9320

In [44]:


dataF = pd.DataFrame(data=trajets, columns=['lat', 'long', 'time', 'cluster'])
csv = dataF.to_csv(path_or_buf="../data/clustering_soiree_4c.csv")